In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
#data analysis
import pandas as pd 
import numpy as np 

#visualising 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches
import shap

#ml
import lightgbm as lgb
import lightgbm as LGBMRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, AdaBoostClassifier 
from sklearn.datasets import make_regression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("diagnosis-of-covid-19-and-its-clinical-spectrum.csv")
# Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')
# Drop the null rows
# df = df.dropna()
df.columns = [x.lower().strip().replace(' ','_') for x in df.columns]

In [17]:
# Number of unique classes in each object column
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

patient_id                                            5644
sars_cov_2_exam_result                                   2
patient_addmited_to_regular_ward_1_yes_0_no              2
patient_addmited_to_semi_intensive_unit_1_yes_0_no       2
patient_addmited_to_intensive_care_unit_1_yes_0_no       2
respiratory_syncytial_virus                              2
influenza_a                                              2
influenza_b                                              2
parainfluenza_1                                          2
coronavirusnl63                                          2
rhinovirus_enterovirus                                   2
coronavirus_hku1                                         2
parainfluenza_3                                          2
chlamydophila_pneumoniae                                 2
adenovirus                                               2
parainfluenza_4                                          2
coronavirus229e                                         

In [20]:
#first ten correlations between the features in dataset
features = df.columns.values[2:112]
corrs_ = df[features].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs_ = corrs_[corrs_['level_0'] != corrs_['level_1']]
#corrs_.head(10)

In [26]:
#Encoding variables: find a way to encode (represent) these variables as numbers before handing them off to the model
#fill in mean for floats
for c in df.columns:
    if df[c].dtype=='float16' or  df[c].dtype=='float32' or  df[c].dtype=='float64':
        df[c].fillna(df[c].mean())
#fill in -999 for categoricals
df = df.fillna(-999)
# Label Encoding
for f in df.columns:
    if df[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[f].values))
        df[f] = lbl.transform(list(df[f].values))      
#print('Labelling done.')  

In [29]:
#remove collinear varialbe: variables that highly correlated to one another 
#can decrease model's availability to learn, decrease model interpretability and 
#decrease generalization performance on the test set
# Threshold for removing correlated variables
threshold = 0.92
# Absolute value correlation matrix
corr_matrix = df.corr().abs()
corr_matrix.head()
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
#upper.head()

In [30]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
# print('There are %d columns to remove.' % (len(to_drop)))
dataset = df.drop(columns = to_drop)
# print('Data shape: ', dataset.shape)
# print('Size of the data', dataset.shape)

In [32]:
#remove missing values
#dataset missing values (in percent)
dataset_missing = (dataset.isnull().sum() / len(dataset)).sort_values(ascending = False)
#dataset_missing.head()

In [33]:
#identify missing values above threshold
dataset_missing_ = dataset_missing.index[dataset_missing > 0.85]
all_missing = list(set(dataset_missing_))
#print('There are %d columns with more than 85%% missing values' % len(all_missing))
dataset = dataset.drop(columns = all_missing)
#print('Data shape: ', dataset.shape)

In [38]:
#feature selection through feature importance
cat_features = [i for i in dataset.columns if str(dataset[i].dtype) in ['object', 'category']]

if len(cat_features) > 0:
    dataset[cat_features] = dataset[cat_features].astype('category')


df_lgb = dataset.copy()
for i in cat_features:
    df_lgb[i] = dataset[i].cat.codes

df_lgb.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_lgb.columns]

dataset_labels = df_lgb['sars_cov_2_exam_result']
df_lgb_ = df_lgb.copy()
df_lgb = df_lgb.drop(['patient_id', 
                      'sars_cov_2_exam_result', 
                      'patient_addmited_to_regular_ward_1_yes_0_no',
                      'patient_addmited_to_semi_intensive_unit_1_yes_0_no',
                      'patient_addmited_to_intensive_care_unit_1_yes_0_no'
                ], axis=1)
x = df_lgb.copy()

In [39]:
# Initialize an empty array to hold feature importances
feature_importances = np.zeros(df_lgb.shape[1])

# Create the model with several hyperparameters
model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 5000, class_weight = 'balanced')

# Fit the model twice to avoid overfitting
for i in range(2):
    # Split into training and validation set
    dataset_features, valid_features, dataset_features_y, valid_y = train_test_split(x, dataset_labels, test_size = 0.20, random_state = i)
    
    # Train using early stopping
    model.fit(dataset_features, dataset_features_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
              eval_metric = 'auc', verbose = 200)
    
    # Record the feature importances
    feature_importances += model.feature_importances_

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[25]	valid_0's auc: 0.649972	valid_0's binary_logloss: 0.610572
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's auc: 0.603197	valid_0's binary_logloss: 0.643876


In [40]:
#average feature importances! 
feature_importances = feature_importances / 2
feature_importances = pd.DataFrame({'feature': list(df_lgb.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
feature_importances.head()

,feature,importance
0,patient_age_quantile,276.5
5,neutrophils,45.5
7,proteina_c_reativa_mg_dl,33.5
1,hematocrit,29.0
3,respiratory_syncytial_virus,26.5


In [41]:
# Find the features with zero importance
zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
#print('There are %d features with 0.0 importance' % len(zero_features))
#feature_importances.tail()

In [42]:
def plot_feature_importances(df, threshold = 0.9):

    plt.rcParams['font.size'] = 18
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])
    importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
#     print('%d features required for %0.2f of cumulative importance' % (importance_index + 1, threshold))
    
    return df

In [59]:
norm_feature_importances = plot_feature_importances(feature_importances)

In [60]:
#remove the features that have zero importance.
df_lgb = df_lgb.drop(columns = zero_features)
#print('Dataset shape: ', df_lgb.shape)

In [43]:
#re-run the model to see if it identifies any more features with zero importance, (kind of like manual recursive feature eilmination)

def identify_zero_importance_features(train, train_labels, iterations = 2):
    #Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])
    #Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')
    #Fit the model multiple times to avoid overfitting
    for i in range(iterations):
        #Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)
        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
                  eval_metric = 'auc', verbose = 200)
        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)  
    # Find the features with zero importance
    zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    return zero_features, feature_importances

In [44]:
#zero_features

In [45]:
second_round_zero_features, feature_importances = identify_zero_importance_features(df_lgb, dataset_labels)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[21]	valid_0's auc: 0.644206	valid_0's binary_logloss: 0.616384
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[39]	valid_0's auc: 0.646297	valid_0's binary_logloss: 0.637929

There are 19 features with 0.0 importance


In [48]:
norm_feature_importances = plot_feature_importances(feature_importances, threshold = 0.95)

In [49]:
# # Threshold for cumulative importance
threshold = 0.95
# Extract the features to keep
features_to_keep = list(norm_feature_importances[norm_feature_importances['cumulative_importance'] < threshold]['feature'])
features_to_keep.append('patient_addmited_to_intensive_care_unit_1_yes_0_no')
features_to_keep.append('patient_addmited_to_regular_ward_1_yes_0_no')
features_to_keep.append('patient_addmited_to_semi_intensive_unit_1_yes_0_no')
features_to_keep.append('patient_id')
features_to_keep.append('sars_cov_2_exam_result')
# Create new datasets with smaller features
dataset_small = df_lgb_[features_to_keep]

In [50]:
train_df = dataset_small
features = list(train_df)
features.remove('patient_id')
features.remove('sars_cov_2_exam_result')
features.remove('patient_addmited_to_intensive_care_unit_1_yes_0_no')
features.remove('patient_addmited_to_regular_ward_1_yes_0_no')
features.remove('patient_addmited_to_semi_intensive_unit_1_yes_0_no')
target = 'sars_cov_2_exam_result'

In [51]:
# Split Train and Validation
X_train = train_df.drop('patient_id',axis=1)
X_train = X_train.drop(['sars_cov_2_exam_result',
                        'patient_addmited_to_regular_ward_1_yes_0_no',
                        'patient_addmited_to_semi_intensive_unit_1_yes_0_no',
                        'patient_addmited_to_intensive_care_unit_1_yes_0_no'],
                       axis=1)
target = train_df['sars_cov_2_exam_result']

X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  target,
                                                  test_size=0.30, 
                                                  random_state=2020, 
                                                  stratify=target)

In [215]:
#model selection
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    model = classifier.fit(X_train, y_train)
    print(classifier)
    print("model score: %.3f" % model.score(X_val, y_val))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
model score: 0.874
SVC(C=0.025, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
model score: 0.901
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
model score: 0.888
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
     

In [52]:
#looking at Random Forest Classifier alone 
rf = RandomForestClassifier(n_estimators=2000) 
rf = rf.fit(X_train, y_train) 
acc = rf.score(X_val, y_val) 
print(f"Training Data Score: {rf.score(X_train, y_train)}") 
print(f"Testing Data Score: {rf.score(X_val, y_val)}")

Training Data Score: 0.9167088607594936
Testing Data Score: 0.9002361275088547


In [53]:
#Hyperparameter optimization aka find the best set of parameters for the algorithm 
#create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,20,30],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                             n_jobs = -1, verbose = 2)

In [56]:
# Train the model with GridSearch
grid_search.fit(X_train, y_train)
grid_search.best_params_
best_grid = grid_search.best_estimator_
print(f"Training Data Score: {best_grid.score(X_train, y_train)}")
print(f"Testing Data Score: {best_grid.score(X_val, y_val)}")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 10.7min finished


Training Data Score: 0.9055696202531646
Testing Data Score: 0.9008264462809917


In [57]:
from sklearn.metrics import classification_report

In [58]:
#use the best model to predict labels on the test set and print classification report 
y_pred_best = best_grid.predict(X_train)
print(classification_report(y_train, y_pred_best))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      3559
           1       1.00      0.05      0.09       391

    accuracy                           0.91      3950
   macro avg       0.95      0.52      0.52      3950
weighted avg       0.91      0.91      0.86      3950



# Save the Model

In [66]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'best_model_rf.sav'
# joblib.dump(best_model_rf, filename)
# filename1 = 'best_model_gnb.sav'
# joblib.dump(best_model_gnb, filename1)

In [70]:
#for dash app figure 
fig_feature_importances = feature_importances[:18]
fig_feature_importances

,feature,importance
0,patient_age_quantile,1238.0
3,respiratory_syncytial_virus,130.0
8,influenza_b_rapid_test,92.0
4,neutrophils,64.5
1,hematocrit,61.5
6,proteina_c_reativa_mg_dl,58.5
5,urea,38.0
13,strepto_a,26.5
7,potassium,17.5
2,serum_glucose,15.5


In [77]:
fig_feature_importances['log10_value'] = np.log10(fig_feature_importances.loc[:,'importance']) 
log_features = fig_feature_importances
log_features

,feature,importance,log10_value
0,patient_age_quantile,1238.0,3.092721
3,respiratory_syncytial_virus,130.0,2.113943
8,influenza_b_rapid_test,92.0,1.963788
4,neutrophils,64.5,1.809560
1,hematocrit,61.5,1.788875
6,proteina_c_reativa_mg_dl,58.5,1.767156
5,urea,38.0,1.579784
13,strepto_a,26.5,1.423246
7,potassium,17.5,1.243038
2,serum_glucose,15.5,1.190332


In [78]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=log_features["log10_value"],
            y=log_features["feature"],
            orientation='h'))
fig.update_layout(yaxis=dict(autorange="reversed"))

fig.show()